Importing Libraries

In [188]:
import math
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
# Specify the search query
#infl =["Joe Biden","Leo Messi","Cristiano Ronaldo","Shah Rukh Khan","Mark Zuckerberg","Narendra Modi","Sam Altman","Elon Musk", "Taylor Swift","Donald Trump"]
import json
newd=[]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import praw
from requests import Session
reddit = praw.Reddit(
    client_id="uRKvUgH0mRJqx3KWrni1AA",
    client_secret="MdP0rgI-cGYOwCOIWyBYvO5F0sIZnQ",
    password="CEna14!@",
    user_agent="testscript by u/fakebot3",
    username="Zealousideal_Tie15", check_for_async=True
)
import warnings
warnings.filterwarnings("ignore")

Data Collection

Creating a list of subreddits for the search query

In [113]:
search_query='Taylor Swift'
subreddits = list(reddit.subreddits.search(search_query, limit=None))
ls=[] 
# Print the names of the subreddits
for subreddit in subreddits:
    ls.append(subreddit.display_name)


In [261]:
search_query='Taylor Swift'
params = {
    'q': search_query,
    'sort': 'new',  # You can adjust the sorting method  
}

# Set the number of results you want per request
results_per_request = 100000000

# Set the maximum number of results you want to retrieve
max_results = 10000000000000  # Adjust this value based on your needs

# Initialize an empty list to store the results
all_results12 = []
for j in ls: 
    try:
        for submission in reddit.subreddit(j).search(search_query, limit=None):
            all_results12.append(submission)

            # Check if we have reached the desired number of results
            if len(all_results12) >= max_results:
                break

    except Exception   as e:
        print(f"403 Forbidden error during Reddit API request: {e}")
        # Handle the 403 error as needed, e.g., print a message and continue the loop

403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response
403 Forbidden error during Reddit API request: received 403 HTTP response


In [268]:
print(len(all_results12))
l12=[]
for i in all_results12:
    l12.append([i.title,search_query,i.score,i.num_comments,i.url,i.selftext,i.permalink,i.created_utc,i.clicked,i.id,i.subreddit,i.subreddit_id])
dfl2=pd.DataFrame(l12)
dfl2.drop_duplicates(inplace=True)
print(len(dfl2))
dfl2.rename(columns={0:'submission_title',1:'influencer',2:'submission_score',3:'submission_num_comments',4:'submission_URLs',5:'submission_selftext',6:'submission_permalink',7:'submission_created_utc',8:'submission_clicked',9:'id',10:'subreddit',11:'subreddit_id'},inplace=True)
dfl2['submission_created_date']=pd.to_datetime(dfl2['submission_created_utc'],unit='s').dt.date
#dfl.drop(['submission_created_utc'],axis=1,inplace=True)
dfl2.sort_values(by='submission_created_utc', ascending=False)

dfl2.head(1)

13233
13233


,submission_title,influencer,submission_score,submission_num_comments,submission_URLs,submission_selftext,submission_permalink,submission_created_utc,submission_clicked,id,subreddit,subreddit_id,submission_created_date
0,Taylor Swift Announces International Tour Dates,Taylor Swift,5686,6338,https://www.reddit.com/gallery/14ef2mx,,/r/TaylorSwift/comments/14ef2mx/taylor_swift_announces_international_tour_dates/,1.687277e+09,False,14ef2mx,TaylorSwift,t5_2rlwe,2023-06-20


Data Preprocessing

In [ ]:

def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text


# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [274]:
dfl2['new_submission_title']=dfl2['submission_title'].apply(lambda x:finalpreprocess(x))
dfl2.head(3)

,submission_title,influencer,submission_score,submission_num_comments,submission_URLs,submission_selftext,submission_permalink,submission_created_utc,submission_clicked,id,subreddit,subreddit_id,submission_created_date,new_submission_title
0,Taylor Swift Announces International Tour Dates,Taylor Swift,5686,6338,https://www.reddit.com/gallery/14ef2mx,,/r/TaylorSwift/comments/14ef2mx/taylor_swift_announces_international_tour_dates/,1.687277e+09,False,14ef2mx,TaylorSwift,t5_2rlwe,2023-06-20,taylor swift announce international tour date
1,who is taylor swift's taylor swift?,Taylor Swift,829,258,https://www.reddit.com/r/TaylorSwift/comments/16pmwf6/who_is_taylor_swifts_taylor_swift/,who does blondie love as much as we love her ??? \n\nmy personal guess is lana maybe ? im not sure,/r/TaylorSwift/comments/16pmwf6/who_is_taylor_swifts_taylor_swift/,1.695420e+09,False,16pmwf6,TaylorSwift,t5_2rlwe,2023-09-22,taylor swift taylor swift
2,Taylor Swift Hits Billionaire Status,Taylor Swift,2678,603,https://www.bloomberg.com/graphics/2023-taylor-swift-net-worth-billionaire/?srnd=undefined,,/r/TaylorSwift/comments/17h7gxt/taylor_swift_hits_billionaire_status/,1.698356e+09,False,17h7gxt,TaylorSwift,t5_2rlwe,2023-10-26,taylor swift hit billionaire status


In [279]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]

Topic Modeling

In [278]:
import pickle

import pyLDAvis
import pyLDAvis.gensim_models
from pyLDAvis.gensim import prepare
from nltk.corpus import stopwords
pyLDAvis.enable_notebook()
stop_words=list(set(stopwords.words('english')))
stop_words.extend(['from','taylor','swift','taylor swift','subject', 're'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words]
            for doc in texts]

def func(i,df4):
   
    filterdf=df4[df4['influencer']==i]
    data = filterdf['new_submission_title'].values.tolist()
    data_words = list(sent_to_words(remove_stopwords(data)))# remove stop words

    id2word = corpora.Dictionary(data_words)# Create Corpus
    texts = data_words# Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    num_topics = 10# Build LDA model
    lda_model = gensim.models.LdaModel(corpus,  id2word=id2word, num_topics=5,passes=15)

    vis = prepare(lda_model, corpus=corpus , dictionary=id2word)
    return vis
func('Taylor Swift',df4=dfl2)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.131680  0.004405       1        1  22.490029
0     -0.058584  0.148556       2        1  20.646119
4     -0.076385  0.114777       3        1  20.260880
3     -0.041074 -0.287961       4        1  19.888232
1      0.307723  0.020222       5        1  16.714740, topic_info=      Term        Freq       Total Category  logprob  loglift
42     era  350.000000  350.000000  Default  30.0000  30.0000
3     tour  445.000000  445.000000  Default  29.0000  29.0000
976  round  276.000000  276.000000  Default  28.0000  28.0000
34    song  494.000000  494.000000  Default  27.0000  27.0000
582  vinyl  230.000000  230.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
264    new   89.161168  258.267273   Topic5  -4.6122   0.7253
51     fan   70.038509  204.228351   Topic5  -4.8536   0.7187
681    sub   37.238354   51.674104   Topic5  -5.4853   1.4613
269   post   37.326530  109.750287   Topic5  -5.4830   0.7104
294    say   38.753503  160.116375   Topic5  -5.4455   0.3702

[259 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
270       1  0.982723  actually
262       3  0.992995       add
22        1  0.091899     album
22        2  0.151363     album
22        4  0.754111     album
...     ...       ...       ...
9         4  0.994751      year
568       5  0.978591      york
2164      1  0.189950     youre
2164      2  0.009498     youre
2164      4  0.797791     youre

[302 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 5, 4, 2])